# Check our calculation of all gradients

In [1]:
import numpy as np
import scipy as sp
#import processData
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from scipy import sparse as spa
from scipy import stats
from scipy.optimize import check_grad
import time

In [2]:
nb_epochs = 300
k = 4
learning_rate = 0.05

In [3]:
def predictionForOne(x, x2, w0, W, V_T, V2_T):
    res1 = np.sum(V_T.dot(x) ** 2)
    res2 = np.sum(V2_T.dot(x2))
    return w0 + W.dot(x) + 0.5 * (res1 - res2)

'''
Generates prediction y for given X, w0, W, V following the definition of factorization machine
'''
def prediction(X,w0,W,V):
    n, m = X.shape
    X2 = X ** 2
    V_T = V.T
    V2_T = (V**2).T
    return [predictionForOne(X[i], X2[i], w0, W, V_T, V2_T) for i in range(n)]

def L(Y, prediction, reg_weight = 0.01,loss = "squared"):
    if(loss == "squared"):
        n = Y.size
        loss = np.sum((Y - prediction) ** 2)/n
        regularizer = reg_weight * W.dot(W)
        return loss + regularizer

def crtness(Y, prediction):
    n = Y.size
    prediction = np.round(prediction)
    prediction = prediction.astype(int)
    return (n - np.count_nonzero(Y - prediction)) / float(n)

def softcrtness(Y, prediction):
    n = Y.size
    prediction = np.round(prediction)
    prediction = prediction.astype(int)
    soft_crt = 0
    for i in range(len(Y)):
        if(abs(Y[i] - prediction[i]) <= 1):
            soft_crt += 1
    return soft_crt/float(n)

def avgdist(Y, prediction):
    n = Y.size
    return np.mean(np.abs(Y - prediction))

def evaluateGradient2(X, w0, W, V, Y, k, reg_weight = 0.05):
    Y = np.array(Y)
    n,m = X.shape
    pred = prediction(X, w0, W, V)
    pred = np.array(pred).flatten()
    regularizer = reg_weight * W.dot(W)
    diff = Y - pred

    dLdw0 = - np.sum(diff) * 2 / n
    dLdWi = -2.0/n * (diff.dot(X)).flatten() + 2 * reg_weight * W
    
    start_time = time.time() # evaluate running time

    dLdV = np.zeros((m, k))
    XV = X.dot(V) 
    for i in range(m):
        temp = np.array(X[:,i]).reshape(-1)
        temp2 = temp ** 2
        for f in range(k):
            v_ifX = V[i][f] * temp2
            Xv_ifX = temp * (XV[:,f].reshape(-1))
            dLdV[i][f] = diff.dot(v_ifX - Xv_ifX)
    dLdV = dLdV * 2.0 / n
    return dLdw0, dLdWi, dLdV

## Check gradient for w0

In [4]:
X = np.array([
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0.5, 0.5],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0.5, 0, 0.5],
    [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0.5, 0.5, 0]
    ])
n, m = X.shape
Y = np.array([5, 3, 1, 4, 5, 1, 5])
W  = np.random.rand(m)
k = 4
V  = np.random.rand(m, k)

def funw0(w0, X = X, Y = Y, W = W, k = k, reg_weight = 0.01,loss = "squared"):
    pred = prediction (X, w0[0], W, V)
    if(loss == "squared"):
        n = Y.size
        loss = np.sum((Y - pred) ** 2)/n
        regularizer = reg_weight * W.dot(W)
        return loss + regularizer
    
def gradw0 (w0, X = X, Y = Y, W = W, k = k, reg_weight = 0.05):
    Y = np.array(Y)
    n,m = X.shape
    pred = prediction(X, w0[0], W, V)
    pred = np.array(pred).flatten()
    regularizer = reg_weight * W.dot(W)
    diff = Y - pred
    dLdw0 = - np.sum(diff) * 2 / n
    return [dLdw0]


In [5]:
w0 = np.random.rand()
errw0 = check_grad (funw0, gradw0, [w0])
print(errw0)

1.38807996208e-07


## Check gradient for W

In [6]:
X = np.array([
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0.5, 0.5],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0.5, 0, 0.5],
    [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0.5, 0.5, 0]
    ])
n, m = X.shape
Y = np.array([5, 3, 1, 4, 5, 1, 5])
w0 = np.random.rand()
k = 4
V  = np.random.rand(m, k)
    
def funwi(W, w0 = w0, X = X, Y = Y, k = k, reg_weight = 0.01,loss = "squared"):
    pred = prediction (X, w0, W, V)
    if(loss == "squared"):
        n = Y.size
        loss = np.sum((Y - pred) ** 2)/n
        regularizer = reg_weight * W.dot(W)
        return loss + regularizer

def gradwi (W, w0 = w0,X = X, Y = Y, k = k, reg_weight = 0.01):
    Y = np.array(Y)
    n,m = X.shape
    pred = prediction(X, w0, W, V)
    pred = np.array(pred).flatten()
    regularizer = reg_weight * W.dot(W)
    diff = Y - pred
    dLdWi = -2.0/n * (diff.dot(X)).flatten() + 2 * reg_weight * W
    return dLdWi

In [7]:
W  = np.random.rand(m)
errwi = check_grad (funwi, gradwi, W)
print(errwi)

1.61977677594e-06


## Check gradient for V

In [24]:
X = np.array([
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0.5, 0.5],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0.5, 0, 0.5],
    [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0.5, 0.5, 0]
    ])
W  = np.random.rand(m)
n, m = X.shape
Y = np.array([5, 3, 1, 4, 5, 1, 5])
w0 = np.random.rand()
k = 4
    
def funV(V, w0 = w0, X = X, Y = Y, k = k, W = W, reg_weight = 0.01,loss = "squared"):
    V = V.reshape(m,k)
    pred = prediction (X, w0, W, V)
    if(loss == "squared"):
        n = Y.size
        loss = np.sum((Y - pred) ** 2)/n
        regularizer = reg_weight * W.dot(W)
        return loss + regularizer

def gradV (V, w0 = w0, m = m, X = X, Y = Y, k = k, W = W, reg_weight = 0.01):
    V = V.reshape(m,k)
    Y = np.array(Y)
    n,m = X.shape
    pred = prediction(X, w0, W, V)
    pred = np.array(pred).flatten()
    diff = Y - pred
    regularizer = reg_weight * W.dot(W)
    dLdV = np.zeros((m, k))
    XV = X.dot(V) 
    for i in range(m):
        temp = np.array(X[:,i]).reshape(-1)
        temp2 = temp ** 2
        for f in range(k):
            v_ifX = V[i][f] * temp2
            Xv_ifX = temp * (XV[:,f].reshape(-1))
            dLdV[i][f] = diff.dot(v_ifX - Xv_ifX)
    dLdV = dLdV * 2.0 / n
    dLdV = dLdV.reshape(-1)
    return dLdV

def gradVVec(V, w0 = w0, m = m, X = X, Y = Y, k = k, W = W, reg_weight = 0.01):

    V = V.reshape(m,k)
    Y = np.array(Y)
    n,m = X.shape
    pred = prediction(X, w0, W, V)
    pred = np.array(pred).flatten()
    regularizer = reg_weight * W.dot(W)
    diff = Y - pred

    dLdw0 = - np.sum(diff) * 2 / n
    dLdWi = -2.0/n * (diff.dot(X)).flatten() + 2 * reg_weight * W
    
    XVT = X.dot(V).T
    xT = X.T
    # xT2 = xT ** 2
    diffT = diff[:, np.newaxis]
    # v_ifX = np.empty((k,n))
    # Xv_ifX = np.empty((k,n))

    x2 = X ** 2

    dLdV = np.array([])
    times = 10
    for i in range(times):
        if i < times - 1:
            v_ifXDiff = np.einsum('ij, jk -> jki', x2[i*int(n/times):(i+1)*int(n/times),:], V[:,i*int(k/times):(i+1)*int(k/times)])
            Xv_ifX = np.einsum('ij, kj -> ikj', xT[:,i*int(n/times):(i+1)*int(n/times)], XVT[i*int(k/times):(i+1)*int(k/times),i*int(n/times):(i+1)*int(n/times)])
            v_ifXDiff -= Xv_ifX
            dLdV = np.append(np.einsum('ijk , kl ->ijl', v_ifXDiff, diffT[i*int(n/times):(i+1)*int(n/times)]), dLdV)
#             dLdV.append(np.einsum('ijk , kl ->ijl', v_ifXDiff, diffT[i*int(n/times):(i+1)*int(n/times)]))
        else:
            v_ifXDiff = np.einsum('ij, jk -> jki', x2[i*int(n/times):,:], V[:,i*int(k/times):])
            Xv_ifX = np.einsum('ij, kj -> ikj', xT[:,i*int(n/times):], XVT[i*int(k/times):,i*int(n/times):])
            v_ifXDiff -= Xv_ifX
            dLdV = np.append(np.einsum('ijk , kl ->ijl', v_ifXDiff, diffT[i*int(n/times):]), dLdV)
#             dLdV.append(np.einsum('ijk , kl ->ijl', v_ifXDiff, diffT[i*int(n/times):]))

    dLdV *= 2/n
    dLdV = dLdV.reshape(-1)

    
#     start_time = time.time() # evaluate running time

#     dLdV = np.zeros((m, k))
#     XV = X.dot(V)
#     xT = X.T
#     xT2 = xT ** 2
#     diffT = diff.reshape(n,1)
#     for i in range(m):
#         v_ifX = (V[i, np.newaxis].T).dot(xT2[i, np.newaxis])
# #         v_ifX = V[i].reshape(k,-1) * xT2[i]
#         v_ifX = (np.expand_dims(V[i], axis = 0).T).dot(np.expand_dims(xT2[i], axis = 0))
# #         v_ifX = V[i].reshape(k,1) * np.tile(xT2[i],(k, 1)) 
# #         if i == 0:
# #             print(V[i].shape)
# #             print(xT2[i].shape)
# #             print((V[i].reshape(k,1)).shape)
# #             print(np.tile(xT2[i],(k, 1)).shape)
# #             print ( "v_ifX's shape is: (%d, %d)" % (v_ifX.shape[0], v_ifX.shape[1]))
#         Xv_ifX = ((xT[i]).reshape(n,-1) * XV).T
#         res = (v_ifX - Xv_ifX).dot(diffT)
#         dLdV[i] = res.ravel()
#     dLdV = dLdV * 2.0 / n
#     dLdV = dLdV.reshape(-1)
    return dLdV

In [25]:
V  = np.random.rand(m * k)
errV = check_grad (funV, gradV, V)
print(errV)
errV = check_grad (funV, gradVVec, V)
print(errV)

1.868001727e-06
1.86800172695e-06
